# Цикл обработки событий

Как уже говорилось раньше, по-умолчанию сокеты прослушивают порт в блокирующем режиме, т.е. выполнение программы блокируется сокетом до появления каких-то данных на прослушиваемом порту. Это имеет очевидное следствие - если мы хотим одновременно обрабатывать несколько соединений, то должны использовать либо процессы либо потоки. Из этого, в свою очередь, следует, что рано или поздно нам придется позаботиться о синхронизации процессов и особенно потоков, что зачастую бывает не самой простой процедурой и чревато ошибками, которые трудно локализовать. 

Данное затруднение породило желание упростить работу программы, пусть даже и за счет потери многопоточности. Таким упрощением является обработка событий в специальном цикле, аналогичном тому, как программы с графическим пользовательским интерфейсом в различных ОС используют цикл обработки пользовательских событий. На наиболее низком уровне такой цикл достигается с использованием библиотеки `select`. Про более высокие уровни будет написано позже. 

Посмотрим пример использования библиотеки `select` для реализации цикла обработки событий в сетевом приложении.

In [22]:
%%writefile non_block_io_server.py

import socket
import select

sock = socket.socket()
sock.bind(("", 10001))
sock.listen()

# Как обрабатывать запросы для conn1 и conn2,
# избегая блокировки выполнения потоков и 
# без использования многопоточности?
conn1, addr = sock.accept()
conn2, addr = sock.accept()

conn1.setblocking(0)
conn2.setblocking(0)

# Создаем очередь входящих сообщений, предусматривая идентификацию соединения, 
# в рамках которого то или иное сообщение было получено
epoll = select.epoll()
epoll.register(conn1.fileno(), select.EPOLLIN | select.EPOLLOUT)
epoll.register(conn2.fileno(), select.EPOLLIN | select.EPOLLOUT)
conn_map = {conn1.fileno(): conn1, conn2.fileno(): conn2}

#А вот и сам цикл обработки сообщений
while True:
    events = epoll.poll(1)
    for fileno, event in events:
        if event & select.EPOLLIN:
            print('data received...')
            data = conn_map[fileno].recv(1024)
            print(data.decode('utf8'))
        elif event & select.EPOLLOUT:
            conn_map[fileno].send('ping'.encode('utf8'))


Overwriting non_block_io_server.py


Теперь для того, чтобы протестировать приведенный выше код сервера, надо запустить его в командной строке, а еще в 2х командных строках надо в интерактивном интерпретаторе Python3 запустить примерно вот такой код:

```python
import socket
import sys

sock = socket.create_connection(("127.0.0.1", 10001))
sock.sendall(b'Some message...')
```